# Ciencia de datos (CC408) - Primavera 2024

## Trabajo Práctico 1 - Parte 2

### Reglas de formato y presentation
- El trabajo debe tener una extensión máxima de 8 páginas (no se permite Apéndice). Se espera una buena redacción en la resolución del práctico.
- El informe debe ser entregado en formato PDF, con los gráficos e imágenes en este mismo archivo.
- Entregar el código con los comandos utilizados, identificando claramente a qué inciso corresponde cada comando.

### Fecha de entrega:
Miércoles 2 de octubre a las 23:59 hs.

### Modalidad de entrega
- Al finalizar el trabajo práctico deben hacer un último <i>commit</i> en su repositorio de GitHub llamado “Entrega final del tp”. 
- Asegurense de haber creado una carpeta llamada TP1. Este Jupyter Notebook y el correspondiente al TP1 - Parte 2 deben estar dentro de esa carpeta.
- También deben enviar el link de su repositorio -para que pueda ser clonado y corregido- al siguiente email: ispiousas@udesa.edu.ar
- La última versión en el repositorio es la que será evaluada. Por lo que es importante que: 
    - No envien el correo hasta no haber terminado y estar seguros de que han hecho el <i>commit y push</i> a la versión final que quieren entregar. Debido a que se pueden tomar hasta 3 días de extensión a lo largo del curso, no se corregirán sus tareas hasta no recibir el correo.
    - No hagan nuevos <i>push</i> despues de haber entregado su versión final. Esto generaría confusión acerca de que versión es la que quieren que se les corrija. 

### Parte I: Limpieza de la base

#### Ejercicio 1
La base de datos provista contiene información sobre oferentes de Airbnb en la ciudad de Nueva York. Realicen una limpieza de la base.
- Tengan en cuenta si hay valores duplicados y eliminénlos.
- Eliminen las columnas que no tienen información de interés.
- (Opcional) Luego de leer el artículo __[Missing-data imputation](https://bookdown.org/mike/data_analysis/imputation-missing-data.html##imputation-missing-data)__ (__[Missing-data imputation articulo alternativo](http://www.stat.columbia.edu/~gelman/arm/missing.pdf)__), decidan qué hacer con los missing values e implementen su decision. Justifiquen su eleccion. Pueden usar informacion del paper mencionado o de otras fuentes (citenlas si las usan) para explicar los problemas que pueden surgir por su estrategia elegida.
- Si hay observaciones con *outliers* o valores que no tienen sentido, tomen una decision. Expliquen las decisiones tomadas.
- Transformen las variables **‘neighbourhood_group’** y **‘room_type’** a variables numericas.
- Con la ayuda de los comandos <code>groupby</code> y <code>join</code> o <code>merge</code>, creen una columna que tenga la cantidad de oferentes por “Neighbourhood group”. Llamen a esa nueva columna <code>offer group</code>

In [2]:
# Resolver acá
import pandas as pd
import os # Exportamos este módulo para trabajar con carpetas
import csv

base_dir = os.getcwd() # Construye la ruta relativa al archivo CSV desde el directorio actual
csv_path = os.path.join(base_dir, 'Base Airbnb NY.csv')

df_airbnb = pd.read_csv(csv_path)
df_airbnb

df_airbnb=df_airbnb.drop(columns=['id', 'name', 'host_id', 'host_name', 'neighbourhood', 'last_review'])
df_airbnb.head(3)

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,149.0,1,9,0.21,6,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,225.0,1,45,0.38,2,355
2,Manhattan,40.80902,-73.94190,Private room,150.0,3,0,NaN,1,365


In [4]:
print(df_airbnb.isnull().sum())

df_airbnb.fillna(df_airbnb.select_dtypes(include=['number']).median(), inplace=True) #Reemplazamos datos nulos con la mediana
print(df_airbnb.isnull().sum())

neighbourhood_group                   0
latitude                              0
longitude                             0
room_type                             0
price                                15
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10053
calculated_host_listings_count        0
availability_365                      0
dtype: int64
neighbourhood_group               0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


In [5]:
# CAMBIO DE VARIABLE A NUMERICA (HAY QUE VER PORQUE NO VA A FUNCIONAR LA ORDINALIDAD
neighbourhood_mapping = {neighbourhood: idx for idx, neighbourhood in enumerate(df_airbnb['neighbourhood_group'].unique())}
df_airbnb['neighbourhood_group'] = df_airbnb['neighbourhood_group'].map(neighbourhood_mapping)

# Mapeo manual para 'room_type'
room_mapping = {room_type: idx for idx, room_type in enumerate(df_airbnb['room_type'].unique())}
df_airbnb['room_type'] = df_airbnb['room_type'].map(room_mapping)

# Mostrar los mapeos
print("Mapping de 'neighbourhood_group':", neighbourhood_mapping)
print("Mapping de 'room_type':", room_mapping)

Mapping de 'neighbourhood_group': {'Brooklyn': 0, 'Manhattan': 1, 'Queens': 2, 'Staten Island': 3, 'Bronx': 4}
Mapping de 'room_type': {'Private room': 0, 'Entire home/apt': 1, 'Shared room': 2}


In [ ]:
# SI SACAMOS OUTLIERS DIRECTAMENTE (HAY QUE VER SI LOS SACAMOS O LO DEJAMOS)

df_airbnb_cleaned = df_airbnb[~df_airbnb.index.isin(outliers_multiple_indices)]

# Verificar el tamaño del nuevo DataFrame
print(f"Número de filas después de eliminar outliers: {df_airbnb_cleaned.shape[0]}")

In [6]:
# Cantidad de oferentes por “Neighbourhood group”.

offer_counts = df_airbnb.groupby('neighbourhood_group').size().reset_index(name='offer_group')

# Paso 2: Unir los conteos al DataFrame original
df_airbnb = df_airbnb.merge(offer_counts, on='neighbourhood_group', how='left')

# Mostrar el DataFrame resultante
print(df_airbnb[['neighbourhood_group', 'offer_group']].head())

   neighbourhood_group  offer_group
0                    0        20107
1                    1        21668
2                    1        21668
3                    0        20107
4                    1        21668


### Parte II: Graficos y visualizaciones

#### Ejercicio 2
Una vez hecha la limpieza, realicen una matriz de correlación con las siguientes variables:**‘neighbourhood group’, ‘latitude’, ‘longitude’, ‘room type’,‘price’, ‘minimum nights’, ‘number of reviews’, ‘reviews per month’,‘calculated host listings count’, ‘availability 365’.** Comenten los resultados. Utilicen alguno de los comandos disponibles en este link para graficar la matriz de correlacion.

*Nota:* consideren cómo es conveniente incluir las variables que originalmente eran categóricas para poder interpretar mejor la matriz de correlación.

In [8]:
# Resolver acá 
# EL PROBLEMA DE ESTE ES QUE LAS VARIABLES ANTES CATEGORICAS AHORA NUMERICAS SE ORDENAN. HAY Q EVITAR ESO

 # Reemplaza con tus columnas numéricas
print(df_airbnb.corr())

                                neighbourhood_group  latitude  longitude  \
neighbourhood_group                        1.000000  0.532846   0.281686   
latitude                                   0.532846  1.000000   0.084761   
longitude                                  0.281686  0.084761   1.000000   
room_type                                 -0.004371 -0.002694  -0.160589   
price                                      0.002701  0.033947  -0.149985   
minimum_nights                            -0.000739  0.024861  -0.062730   
number_of_reviews                          0.011476 -0.015368   0.059063   
reviews_per_month                          0.085852 -0.014071   0.137126   
calculated_host_listings_count             0.044511  0.019514  -0.114699   
availability_365                           0.121283 -0.010602   0.080341   
offer_group                               -0.728394 -0.038169  -0.573156   

                                room_type     price  minimum_nights  \
neighbourhood_gr

#### Ejercicio 3
Respondan las siguientes preguntas: ¿Cuál es la proporción de oferentes por “Neighbourhood group”? ¿Y por tipo de habitación? Además, realicen gráficos para mostrar estas composiciones y comenten los resultados.

In [ ]:
# Resolver acá


#### Ejercicio 4
Realicen un histograma de los precios de los alojamientos. Comenten el gráfico obtenido. Además, respondan las siguientes preguntas: ¿cuál es el precio mínimo, máximo y promedio? ¿Cuál es la media de precio por “Neighbourhood group” y por tipo de habitación? 

In [ ]:
# Resolver acá


#### Ejercicio 5
Realicen dos scatter plots con dos variables de interés en cada uno. Comenten.

In [1]:
# Resolver acá


#### Ejercicio 6
Utilicen el análisis de componentes principales para graficar las variables en dos dimensiones. Comenten los resultados obtenidos (qué porcentaje de la varianza se logra explicar con dos componentes, cómo son los *loadings*, si ven algún patrón en el gráfico).

In [1]:
# Resolver acá


### Parte III: Predicción

El objetivo de esta parte del trabajo es intentar predecir los precios de los alojamientos.

#### Ejercicio 7
Eliminen de la base todas las variables relacionadas al precio

In [ ]:
# Resolver acá


#### Ejercicio 8
Partan la base en una base de prueba (test) y una de entrenamiento (train) utilizando el comando <code>train_test_split</code>. La base de entrenamiento debe comprender el 70% de los datos, y la semilla a utilizar (*random state instance*) debe ser 201. Establezca a price como su variable dependiente en la base de entrenamiento (vector y). El resto de las variables ser´an las variables independientes (matriz X). Recuerden agregar la columna de unos (1).

In [ ]:
# Resolver acá


#### Ejercicio 9
Implementen una regresión lineal y comenten los resultados obtenidos.

In [ ]:
# Resolver acá
